In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Read data
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'

df_people_2021 = pd.read_csv(pathdata + 'data_input/personas_tot_urb_3T_21.txt', delimiter=';', low_memory=False)
df_people_2022 = pd.read_csv(pathdata + 'data_input/personas_tot_urb_3T_22.txt', delimiter=';', low_memory=False)

df_houses_2021 = pd.read_csv(pathdata + 'data_input/hogar_tot_urb_3T_2021.txt', delimiter=';', low_memory=False)
df_houses_2022 = pd.read_csv(pathdata + 'data_input/hogar_tot_urb_3T_2022.txt', delimiter=';', low_memory=False)

In [8]:

merged_df = pd.merge(df_people_2021[['CODUSU', 'COMPONENTE']], df_people_2022[['CODUSU', 'COMPONENTE']], on=['CODUSU', 'COMPONENTE'], how='outer', indicator=True)

# Duplicate rows and display the left-only rows (2021) and right-only rows (2022)
duplicated_rows = merged_df['_merge'].value_counts()['both']
left_only_rows = merged_df['_merge'].value_counts()['left_only']
right_only_rows = merged_df['_merge'].value_counts()['right_only']

# Select the right-only and right observation from both rows in merged_df
right_only_rows = merged_df[merged_df['_merge'] == 'right_only']
left_only_rows = merged_df[merged_df['_merge'] == 'left_only']
both_rows = merged_df[merged_df['_merge'] == 'both']
right_both_rows = pd.concat([both_rows, right_only_rows])

columns = ['PONDERA', 'PONDII', 'PONDIIO']
df_people = [df_people_2021, df_people_2022]

for col in columns:
    for df in df_people:
        df[f'rel_{col}'] = df[col] / df[col].sum()

df_merged_with_copies = pd.concat([df_people_2021, df_people_2022])

df_people_2021_no_dupl = pd.merge(left_only_rows, df_people_2021, on=["CODUSU", "COMPONENTE"], how='inner')
df_people_2022_no_dupl = pd.merge(right_both_rows, df_people_2022, on=["CODUSU", "COMPONENTE"], how='inner')

df_merged_without_copies = pd.concat([df_people_2021_no_dupl, df_people_2022_no_dupl])

new_columns = ['P21_new', 'P47T_new', 'TOT_P12_new']
df_merged_without_copies[new_columns] = pd.NaT

masks = [
    (df_merged_without_copies['ANO4'] == 2021) & (df_merged_without_copies['P21'] != -9),
    (df_merged_without_copies['ANO4'] == 2021) & (df_merged_without_copies['P47T'] != -9),
    (df_merged_without_copies['ANO4'] == 2021) & (df_merged_without_copies['TOT_P12'] != -9),
    (df_merged_without_copies['ANO4'] == 2022),
]

divisors = [0.546358573, 0.546358573, 0.546358573, 1]

for i, mask in enumerate(masks):
    df_merged_without_copies.loc[mask, new_columns[i]] = df_merged_without_copies.loc[mask, new_columns[i]] / divisors[i]


In [10]:
df_people_2021['H15'].value_counts()

1    67764
0    10354
2       99
Name: H15, dtype: int64

In [15]:
df_people_2021[['H15', 'ESTADO']].value_counts().sort_index()

H15  ESTADO
0    4         10354
1    1         32544
     2          2276
     3         32944
2    0            99
dtype: int64

In [16]:
df_people_2022[['H15', 'ESTADO']].value_counts().sort_index()

H15  ESTADO
0    4         10105
1    1         33202
     2          1982
     3         32234
2    0           119
dtype: int64

In [19]:
df_people_2021['CAT_OCUP'].value_counts().sort_index()

0.0      581
1.0     1302
2.0     8251
3.0    24434
4.0      250
9.0        2
Name: CAT_OCUP, dtype: int64

In [20]:
df_people_2022['CAT_OCUP'].value_counts().sort_index()

0.0      531
1.0     1224
2.0     7765
3.0    25490
4.0      173
9.0        1
Name: CAT_OCUP, dtype: int64

In [22]:
df_people_2021['PP03C'].value_counts().sort_index()

0    46896
1    28566
2     2754
3        1
Name: PP03C, dtype: int64

In [23]:
df_people_2022['PP03C'].value_counts().sort_index()

0    45371
1    29354
2     2917
Name: PP03C, dtype: int64

In [21]:
df_people_2021['PP03D'].value_counts().sort_index()

0.0    29789
2.0     2362
3.0      281
4.0       57
5.0       33
6.0        8
7.0        6
8.0        5
9.0        3
Name: PP03D, dtype: int64

In [40]:
df_people_2021[(df_people_2021['PP3E_TOT'] < df_people_2021['PP3F_TOT'].str.replace(',', '.').astype(float))]['PONDERA'].sum()

130734

In [42]:
df_people_2022[(df_people_2022['PP3E_TOT'] < df_people_2022['PP3F_TOT'])]['PONDERA'].sum()

142258

In [56]:
df_people_2022[df_people_2022['PP3F_TOT']==999]['PP3F_TOT']

28132    999
28656    999
29823    999
29964    999
30335    999
31726    999
Name: PP3F_TOT, dtype: int64

In [58]:
df_people_2022['CAT_INAC'].value_counts().sort_index()

0    35303
1    12288
2      103
3    18257
4     4955
5     5298
6      192
7     1246
Name: CAT_INAC, dtype: int64

In [50]:
df_people_2022['PP3E_TOT'].value_counts()

40.0     5232
48.0     3788
30.0     3184
20.0     2017
25.0     1227
         ... 
140.0       1
102.0       1
119.0       1
108.0       1
83.0        1
Name: PP3E_TOT, Length: 106, dtype: int64

In [24]:
df_people_2022['PP03D'].value_counts().sort_index()

0.0    30285
2.0     2502
3.0      296
4.0       68
5.0       29
6.0       12
7.0        6
8.0        3
9.0        1
Name: PP03D, dtype: int64

In [44]:
df_people_2021['PP07H'].value_counts().sort_index()

0.0     9341
1.0    14631
2.0     8572
Name: PP07H, dtype: int64

In [12]:
import pandas as pd

def row_counts(df_year1, df_year2, year1, year2):
    count_year1 = len(df_year1)
    count_year2 = len(df_year2)
    merged_df = pd.merge(df_year1, df_year2, on=['CODUSU', 'COMPONENTE'], how='inner')
    count_both_years = len(merged_df)

    # Calculate total population
    pob_year1 = df_year1['PONDERA'].sum()
    pob_year2 = df_year2['PONDERA'].sum()

    # Calculate number of surveys performed
    encuesta_realizada_year1 = df_year1[df_year1['H15'] != 2]['H15'].count()
    pob_encuesta_realizada_year1 = df_year1[df_year1['H15'] != 2]['PONDERA'].sum()
    encuesta_realizada_year2 = df_year2[df_year2['H15'] != 2]['H15'].count()
    pob_encuesta_realizada_year2 = df_year2[df_year2['H15'] != 2]['PONDERA'].sum()

    # Calculate number of imputed surveys
    encuesta_imputada_year1 = df_year1[df_year1['IMPUTA'] == 2]['IMPUTA'].count()
    pob_encuesta_imputada_year1 = df_year1[df_year1['IMPUTA'] == 2]['PONDERA'].sum()
    encuesta_imputada_year2 = df_year2[df_year2['IMPUTA'] == 2]['IMPUTA'].count()
    pob_encuesta_imputada_year2 = df_year2[df_year2['IMPUTA'] == 2]['PONDERA'].sum()

    # Create the table as a pandas dataframe
    data = {
        'Anio': [year1, year2],
        'Cantidad de observaciones': [count_year1, count_year2],
        'Observaciones en ambos anios': [count_both_years, count_both_years],
        'Poblacion total': [pob_year1, pob_year2],
        'Encuestas realizadas en el anio': [encuesta_realizada_year1, encuesta_realizada_year2],
        'Poblacion de encuestas realizadas en el anio': [pob_encuesta_realizada_year1, pob_encuesta_realizada_year2],
        'Encuestas imputadas en el anio': [encuesta_imputada_year1, encuesta_imputada_year2],
        'Poblacion de encuestas imputadas en el anio': [pob_encuesta_imputada_year1, pob_encuesta_imputada_year2]
    }
    table = pd.DataFrame(data)
    table.set_index('Anio', inplace=True)

    return table


In [13]:
result_table = row_counts(df_people_2021, df_people_2022, 2021, 2022)
result_table


,Cantidad de observaciones,Observaciones en ambos anios,Poblacion total,Encuestas realizadas en el anio,Poblacion de encuestas realizadas en el anio,Encuestas imputadas en el anio,Poblacion de encuestas imputadas en el anio
Anio,,,,,,,
2021,78217,30417,41693498,78118,41511018,110,41693498
2022,77642,30417,42268056,77523,42191941,109,42268056


In [ ]:

df_people_2021['PONDERA_new'] = df_people_2021['rel_PONDERA']/2 * (pob_2021_pondera + pob_2022_pondera)/2
df_people_2022['PONDERA_new'] = df_people_2022['rel_PONDERA']/2 * (pob_2021_pondera + pob_2022_pondera)/2

df_people_2021['PONDII_new'] = df_people_2021['rel_PONDII']/2 * (pob_2021_pondii + pob_2022_pondii)/2
df_people_2022['PONDII_new'] = df_people_2022['rel_PONDII']/2 * (pob_2021_pondii + pob_2022_pondii)/2

df_people_2021['PONDIIO_new'] = df_people_2021['rel_PONDIIO']/2 * (pob_2021_pondiio + pob_2022_pondiio)/2
df_people_2022['PONDIIO_new'] = df_people_2022['rel_PONDIIO']/2 * (pob_2021_pondiio + pob_2022_pondiio)/2


In [11]:
df_merged_without_copies.to_csv(pathdata + 'data_output/Base_pool_solo_con_replicas_actuales.csv')